To run on GPU activate conda environment (samplestructures) and run jupyter-notebook (not from VS code)

In [1]:
# -*- coding: utf-8 -*-
"""
Created on Mon April 29 13:25:11 2020

@author: rfuchs
"""



import sys

import os 

%matplotlib inline
import pandas as pd
from copy import deepcopy
from gower import gower_matrix
import matplotlib.pyplot as plt
import matplotlib
from sklearn.preprocessing import LabelEncoder 

from src.MIAMI import MIAMI
from src.init_params import dim_reduce_init
from src.data_preprocessing import compute_nj
from src.utils_methods import encode, get_var_metadata, post_process, transform_df_to_json, generate_plots

from shapely.geometry import Polygon as polygon

import autograd.numpy as np

import colorcet as cc
from scipy.spatial.distance import pdist, squareform
from matplotlib.colors import ListedColormap
import seaborn as sns

/home/luser/miniconda3/envs/samplestructures/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


RUNNING ON CPU


In [2]:
import json

main_folder = ''
res_folder = main_folder + "results/DGMM/"

data_folder = "results/"




    
dtypes_dict = {'continuous': float, 'categorical': str, 'ordinal': int,\
              'bernoulli': int, 'binomial': int}
    
dtypes_dict_famd = {'continuous': float, 'categorical': str, 'ordinal': str,\
              'bernoulli': str, 'binomial': str}
#===========================================#
# Importing data
#===========================================#

inf_nb = 1E12

sub_design = "bivariate"

# acceptance_rate =

def find_files_with_inf(directory_path):
    result_list = []

    # Iterate over each folder in the specified directory
    for folder_name in os.listdir(directory_path):
        folder_path = os.path.join(directory_path, folder_name)

        # Check if it's a directory
        if os.path.isdir(folder_path):
            # List all files in the folder
            files_in_folder = os.listdir(folder_path)

            # Filter files with "inf" in their names
            inf_files = [file for file in files_in_folder if "inf" in file]

            # Add the folder path and corresponding "inf" files to the result list
            result_list.append({"folder_path": folder_path, "inf_files": inf_files})

    return result_list


import json
with open(data_folder + "real/real_structures.json") as f:
    real_orig = json.load(f)
# with open(data_folder + "GA/optimal_run_pop_100_gen_2000_plausibility_FAMD GMM.json") as f:
#     ga_2000 = json.load(f)
# with open(data_folder + "GA/optimal_run_pop_100_gen_2000_plausibility_FAMD GMM_weighted_10.json") as f:
#     ga_plaus_famd_gmm_weighted_10 = json.load(f)
# with open(data_folder + "GA/optimal_run_pop_100_gen_2000_plausibility_FAMD GMM_weighted_100.json") as f:
#     ga_plaus_famd_gmm_weighted_100 = json.load(f)
# with open(data_folder + "GA/optimal_run_pop_100_gen_2000_plausibility_FAMD GMM_weighted_1000.json") as f:
#     ga_plaus_famd_gmm_weighted_1000 = json.load(f)
# with open(data_folder + "GA/optimal_run_pop_100_gen_2000_plausibility_FAMD GMM_weighted_10000.json") as f:
#     ga_plaus_famd_gmm_weighted_10000 = json.load(f)

# DATA_ARRAY = [real_orig, ga_2000, ga_plaus_famd_gmm_weighted_10, ga_plaus_famd_gmm_weighted_100, ga_plaus_famd_gmm_weighted_1000, ga_plaus_famd_gmm_weighted_10000]
# DATA_NAMES = ["real", "GA-plausibility (FAMD GMM)", "GA-plausibility (FAMD GMM) - weighted plausibility 10", "GA-plausibility (FAMD GMM) - weighted plausibility 100", "GA-plausibility (FAMD GMM) - weighted plausibility 1000", "GA-plausibility (FAMD GMM) - weighted plausibility 10000"]

# with open("results_final/GA_done_final/ga_results_one class svm_nu_0.5/P[One Class SVM]_thr_inf_seed_0_evaluated.json") as f:
    # ga = json.load(f)

result_ga_lists_inf = find_files_with_inf("results_final/GA_done_final/")

folders_to_load_from = ["results_final/GA_done_final/kde"]
all_ga_inf = []
for res_dict in result_ga_lists_inf:
    folder_path = res_dict["folder_path"]
    if folder_path not in folders_to_load_from:
        continue
    for inf_file in res_dict["inf_files"]:
        with open(os.path.join(folder_path,inf_file)) as f:
            ga = json.load(f)
        all_ga_inf.extend(ga)
print(len(all_ga_inf))


with open(data_folder + "evaluated/GA/P[None]_seed_0_evaluated.json") as f:
    ga_no_plaus = json.load(f)

# DATA_ARRAY = [real_orig, all_ga_inf]
# DATA_NAMES = ["real", "all_ga_inf"]
# DATA_ARRAY = [real_orig, ga_no_plaus]
# DATA_NAMES = ["real", "ga_no_plaus"]
DATA_ARRAY = [real_orig]
DATA_NAMES = ["real"]

1000


In [3]:
from src.plausibility_metrics import *
discriminator = ClassKDE()
real_scores = discriminator.get_real_plaus()
median = np.median(real_scores)
std = np.std(real_scores) 
threshold = median + 1*std

KDE


In [4]:
brace_dict = {    
    "NONE": 0,
    "H": 1,
    "Z": 2,
    "IZ": 3,
    "ZH": 4,
    "IZH": 5,
    "K": 6,
    "X": 7,
    "XH": 8,
    "nan": 9,
}

brace_dict_inv = dict(zip(brace_dict.values(), brace_dict.keys()))

N_BRACES = len(brace_dict)

In [5]:
max_layers = max(d.get('n_layers', 0) for sublist in DATA_ARRAY for d in sublist)
transformed_columns = ["legs", "total_height", "radius_bottom", "radius_top", "n_layers"]
brace_cols = ["brace" + str(i) for i in range(max_layers-1)] 
layer_cols = ["layer_height" + str(i) for i in range(max_layers-2)]
transformed_columns += brace_cols
transformed_columns += layer_cols

dataframes = []
for dataset, name in zip(DATA_ARRAY, DATA_NAMES):
    encoding = [encode(d, max_layers, brace_dict, N_BRACES, one_hot=False, native=True, normalize_layer_heights=True) for d in dataset]
    df_ = pd.DataFrame(encoding, columns=transformed_columns)
    df_["label"] = [name]*len(df_)
    dataframes.append(df_.copy())
    
train_original = pd.concat(dataframes, axis=0, ignore_index=True)

In [6]:
train_original

,legs,total_height,radius_bottom,radius_top,n_layers,brace0,brace1,brace2,brace3,brace4,layer_height0,layer_height1,layer_height2,layer_height3,label
0,4,22.0,16.968,15.554,2,X,nan,nan,nan,nan,1.0,1.0,1.0,1.0,real
1,3,77.0,21.362,11.761,3,X,X,nan,nan,nan,0.5454545454545454,1.0,1.0,1.0,real
2,4,32.9,20.704,6.64,3,X,X,nan,nan,nan,0.5455927051671733,1.0,1.0,1.0,real
3,3,50.0,16.166,8.706,3,X,X,nan,nan,nan,0.48200000000000004,1.0,1.0,1.0,real
4,3,51.0,19.029,9.336,3,X,X,nan,nan,nan,0.5490196078431373,1.0,1.0,1.0,real
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,4,44.5,19.796,7.777,5,X,X,X,X,nan,0.3258426966292135,0.5955056179775281,0.8202247191011236,1.0,real
96,4,43.5,18.382,7.777,5,X,X,X,X,nan,0.3218390804597701,0.593103448275862,0.8160919540229885,1.0,real
97,4,50.3,18.382,7.777,5,X,X,X,X,nan,0.3101391650099404,0.5745526838966203,0.8051689860834991,1.0,real
98,4,52.0,18.382,7.777,5,X,X,X,X,nan,0.21346153846153845,0.4461538461538461,0.7346153846153847,1.0,real


In [7]:
# Concatenate the two dataframes together and reindex
# train_original = pd.concat([real, synthetic, rand], axis=0, ignore_index=True)

# synth_original = pd.concat([synthetic], axis=0, ignore_index=True)

# train_original = pd.concat([real], axis=0, ignore_index=True)

nominal_features = brace_cols
ordinal_features = ["n_layers", "legs"]
BERNOULLI = ["legs"]

discrete_features = nominal_features + ordinal_features


continuous_features = list(set(transformed_columns) - set(nominal_features) - set(ordinal_features))

train_original[ordinal_features] = train_original[ordinal_features].astype("int")
train_original[continuous_features] = train_original[continuous_features].astype("float")

# synth_original[ordinal_features] = synth_original[ordinal_features].astype("int")
# synth_original[continuous_features] = synth_original[continuous_features].astype("float")

train = train_original.drop("label", axis=1)

# synth = synth_original.drop("label", axis=1)





train = train.infer_objects()
# synth = synth.infer_objects()

numobs = len(train)
print("Running with", numobs, "observations!!!!")

Running with 100 observations!!!!


In [9]:

var_distrib, var_transform_only, le_dict, brace_cols, unique_braces = get_var_metadata(train, train_original, brace_cols, BERNOULLI)



nj, nj_bin, nj_ord, nj_categ = compute_nj(train, var_distrib)

nb_cont = np.sum(var_distrib == 'continuous')     

p = train.shape[1]
        
# Feature category (cf)
dtype = {train.columns[j]: dtypes_dict_famd[var_transform_only[j]] for j in range(p)}

train_famd = train.astype(dtype, copy=True)
numobs = len(train)

authorized_ranges = None


In [10]:
var_transform_only

['bernoulli',
 'continuous',
 'continuous',
 'continuous',
 'ordinal',
 'categorical',
 'categorical',
 'ordinal',
 'ordinal',
 'categorical',
 'continuous',
 'continuous',
 'continuous',
 'continuous']

In [11]:
var_distrib

array(['bernoulli', 'continuous', 'continuous', 'continuous', 'ordinal',
       'bernoulli', 'bernoulli', 'ordinal', 'ordinal', 'bernoulli',
       'continuous', 'continuous', 'continuous', 'continuous'],
      dtype='<U10')

In [12]:
train.dtypes

legs               int64
total_height     float64
radius_bottom    float64
radius_top       float64
n_layers           int64
brace0             int64
brace1             int64
brace2             int64
brace3             int64
brace4             int64
layer_height0    float64
layer_height1    float64
layer_height2    float64
layer_height3    float64
dtype: object

In [13]:
train_famd.dtypes

legs              object
total_height     float64
radius_bottom    float64
radius_top       float64
n_layers          object
brace0            object
brace1            object
brace2            object
brace3            object
brace4            object
layer_height0    float64
layer_height1    float64
layer_height2    float64
layer_height3    float64
dtype: object

In [ ]:
print(var_transform_only)
print(var_distrib)

In [ ]:
sub_design = "none"


In [ ]:
#===========================================#
# Model Hyper-parameters
#===========================================#

n_clusters = 3
r = np.array([2, 1])
k = [n_clusters]

seed = 2023
init_seed = 2024
    
# !!! Changed eps
eps = 1E-10
it = 1000
maxstep = 1000
target_nb_pseudo_obs = 100

# print(train_famd)
#*****************************************************************
# Run MIAMI
#*****************************************************************
print("Initialize dimensionality reduction")    
init, transformed_famd_data, famd  = dim_reduce_init(train_famd, n_clusters, k, r, nj, var_distrib, seed = 2023,\
                                use_light_famd=True)

print(init.keys())
print(len(init["eta"][0]))
print(len(init["H"]))

print("Computing distance matrix")
# # Defining distances over the features
# dm = gower_matrix(train, cat_features = cat_features) 

distances = pdist(transformed_famd_data)

dm = squareform(distances)
print("done")

In [ ]:
lambdass = ['lambda_bin', 'lambda_ord', 'lambda_cont', 'lambda_categ']
print(var_distrib)
summ = 0
for lambas in lambdass:
    print(lambas, len(init[lambas]), np.array(init[lambas][0]).size)
    summ += len(init[lambas]) * np.array(init[lambas][0]).size

print(summ)

In [ ]:
plausibility_filter = {"clf": discriminator, "threshold": threshold}

In [ ]:
print("Training MIAMI with", train.shape)
out = MIAMI(train, n_clusters, r, k, init, var_distrib, nj, le_dict, authorized_ranges, discrete_features, dtype, target_nb_pseudo_obs=target_nb_pseudo_obs, it=it,\
                eps=eps, maxstep=maxstep, seed=seed, perform_selec = True, dm = dm, max_patience = 0)
print('MIAMI has kept one observation over', round(1 / out['share_kept_pseudo_obs']),\
        'observations generated')
    
acceptance_rate = out['share_kept_pseudo_obs']
print(acceptance_rate)
pred = pd.DataFrame(out['y_all'], columns = train.columns) 
print(pred.shape)

In [ ]:
def draw_z_s_from_y(y, mu_s, sigma_s, eta, M):
    ''' Generate z_s and zc_s using input data y instead of drawing random values.
    y (ndarray): Input data
    mu_s (list of nd-arrays): The means of the Gaussians starting at each layer
    sigma_s (list of nd-arrays): The covariance matrices of the Gaussians 
                                 starting at each layer
    eta (list of nb_layers elements of shape (K_l x r_{l-1}, 1)): mu parameters
                                                                 for each layer
    M (list of int): The number of MC to generate on each layer
    -------------------------------------------------------------------------
    returns (list of ndarrays): z^{l} | s for all s in Omega and all l in L
    '''

    L = len(mu_s) - 1    
    r = [mu_s[l].shape[1] for l in range(L + 1)]
    S = [mu_s[l].shape[0] for l in range(L + 1)]
    
    z_s = []
    zc_s = []  # z centered (denoted c) or all l

    for l in range(L + 1):
        if l == 0:
            zl_s = y[:, np.newaxis, :, np.newaxis]  # Reshape y for the first layer
        else:
            zl_s = multivariate_normal(size=(M[l], 1),
                                       mean=mu_s[l].flatten(order='C'),
                                       cov=block_diag(*sigma_s[l]))
        zl_s = zl_s.reshape(M[l], S[l], r[l], order='C')
        z_s.append(t(zl_s, (0, 2, 1)))

        if l < L:  # The last layer is already centered
            eta_ = np.repeat(t(eta[l], (2, 0, 1)), S[l + 1], axis=1)
            zc_s.append(zl_s - eta_)

    return z_s, zc_s

In [ ]:
cols = train.columns
layer_cols_ = np.array([i for i, s in enumerate(cols) if s.startswith('layer_height')])
brace_cols_ = np.array([i for i, s in enumerate(cols) if s.startswith('brace')])

pred_post, pred_post_famd = post_process(pred, train.columns, le_dict, discrete_features, brace_cols, dtype, layer_cols_)

In [ ]:
import json

json_output = transform_df_to_json(pred_post)
out_name = "dgmm("+"_".join(DATA_NAMES) + ")_" + str(target_nb_pseudo_obs) + ".json"
output_filename = res_folder + out_name
with open(output_filename, "w") as json_file:
    json.dump(json_output, json_file, indent=4)


In [ ]:
# zz = out["zz"]
# npzz = np.array(zz)
# npzz = np.reshape(npzz, (npzz.shape[0], npzz.shape[-1]))

# mean = np.mean(npzz, axis=0)
# var = np.std(npzz, axis=0)
# var_s = 2


# x_lim = [mean[0] - var_s*var[0], mean[0] + var_s*var[0]]
# y_lim = [mean[1] - var_s*var[1], mean[1] + var_s*var[1]]


def plot_discr(out, train_original, x_lim=[], y_lim=[]):
    df_data = pd.DataFrame()
    df_generated = pd.DataFrame()
    
    df_data["x"] = out['Ez.y'][:,0]
    df_data["y"] = out['Ez.y'][:,1]
    
    zz = out['zz']
#     zz = famd.transform(pred_famd)
    
    df_generated["x"],  df_generated["y"] = zz[:,0], zz[:,1]
    
    df_generated["method"] = "synthetic"
    
    df_data["method"] = train_original["label"]
    
    df_scatter = pd.concat([df_data, df_generated], axis=0, ignore_index=True)

    not_real_inp = df_data[df_data['method'] != 'real']
    real_inp = df_data[df_data['method'] == 'real']

    not_real_all = pd.concat([not_real_inp, df_generated], axis=0, ignore_index=True)

    # Create a scatter plot
    # fig, ax = plt.subplots(figsize=(8, 6))
    # ax = sns.scatterplot(data=df_scatter, x="x", y="y", style="method", hue="method", alpha=0.7)
    # Create a scatter plot

    # Create a KDE plot on the same Axes
    g = sns.displot(data=not_real_all, x="x", y="y", hue="method", alpha=0.5, kind="kde")
    g1 = sns.scatterplot(data=real_inp, x="x", y="y", label="real", color="black", s=60, ax=g.ax)

    # # Get the legends from both plots
    # legend1 = g.ax.get_legend()
    # legend2 = g1.get_legend()

    # # Concatenate the legends
    # all_legends = legend1.legendHandles + legend2.legendHandles

    # # Set the concatenated legend
    # g.ax.legend(all_legends, ['Legend1', 'Legend2'])
    # handles, labels = ax.get_legend_handles_labels()
    # kde_legend = plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='blue', markersize=10, label='KDE Legend')
    # legend = ax.legend(handles=[kde_legend], labels=['KDE Legend'], loc='center left', bbox_to_anchor=(1, 0.5))

    # Add a title and axis labels
    # ax.set_title('Scatter plot grouped by type')
    # ax.set_xlabel('PC1')
    # ax.set_ylabel('PC2')

    # # Create a legend
    # # legend = ax.legend(loc='center left', bbox_to_anchor=(1.05, 0.5), title="Data", borderaxespad=0.)
    # if len(x_lim)>0:
    #     ax.set_xlim(x_lim[0], x_lim[1])
    #     ax.set_ylim(y_lim[0], y_lim[1])

    plt.show()

plot_discr(out, train_original)
# plot_discr(out, train_original, x_lim=x_lim, y_lim=y_lim)

In [ ]:
to_plot = DATA_NAMES + ["synthetic"]
generate_plots(to_plot, train_original, pred_post, out['Ez.y'], out['zz'], var_distrib, le_dict, brace_cols, unique_braces, res_folder, percentage=True)        

In [ ]:
pred_post_famd = pred_post.astype(dtype, copy=True)


In [ ]:
new_zs_famd = famd.transform(pred_post_famd)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
new_famd = pd.DataFrame()
real_famd = pd.DataFrame()


real_famd["x"] = transformed_famd_data[:,0]
real_famd["y"] = transformed_famd_data[:,1]


new_famd["x"] = new_zs_famd[:,0]
new_famd["y"] = new_zs_famd[:,1]

ax = sns.scatterplot(data=new_famd, x="x", y="y")
ax = sns.scatterplot(data=real_famd, x="x", y="y")

ax = sns.scatterplot(data=new_famd.iloc[0:1], x="x", y="y")

ax = sns.scatterplot(data=new_famd.iloc[-2:-1], x="x", y="y")


# Add a title and axis labels
ax.set_title('Scatter plot grouped by type')
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')

In [ ]:
df_data = pd.DataFrame()
df_data["x"] = out['Ez.y'][:,0]
df_data["y"] = out['Ez.y'][:,1]

zz = out['zz']
df_generated = pd.DataFrame()
df_generated["x"],  df_generated["y"] = zz[:,0], zz[:,1]

ax = sns.scatterplot(data=df_data, x="x", y="y")
ax = sns.scatterplot(data=df_generated, x="x", y="y")

ax = sns.scatterplot(data=df_generated.iloc[0:1], x="x", y="y")
ax = sns.scatterplot(data=df_generated.iloc[-2:-1], x="x", y="y")


# Add a title and axis labels
ax.set_title('Scatter plot grouped by type')
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')

In [ ]:
from scipy.spatial.distance import mahalanobis
from scipy.linalg import inv

# Calculate mean and covariance matrix of the data
mean = np.mean(transformed_famd_data, axis=0)
covariance_matrix = np.cov(transformed_famd_data, rowvar=False)

# Calculate the inverse of the covariance matrix
inv_cov_matrix = inv(covariance_matrix)

# New test instance
test_instance1 = np.array(new_zs_famd[0])

test_instance2 = np.array(new_zs_famd[-1])

# Calculate Mahalanobis distance
mahalanobis_distance1 = mahalanobis(test_instance1, mean, inv_cov_matrix)
mahalanobis_distance2 = mahalanobis(test_instance2, mean, inv_cov_matrix)
print(mahalanobis_distance1, mahalanobis_distance2)

In [ ]:
from scipy.linalg import block_diag
mu = out['mu'][0]
sigma = out['sigma'][0]
weights = out['w_s']

In [ ]:
print(mu.shape)
print(sigma.shape)
print(weights.shape)

In [ ]:
import matplotlib.lines as mlines
from matplotlib.patches import Ellipse


def draw_ellipse(position, covariance, ax=None, **kwargs):
    """Draw an ellipse with a given position and covariance"""
    ax = ax or plt.gca()
    
    # Convert covariance to principal axes
    if covariance.shape == (2, 2):
        U, s, Vt = np.linalg.svd(covariance)
        angle = np.degrees(np.arctan2(U[1, 0], U[0, 0]))
        width, height = 2 * np.sqrt(s)
    else:
        angle = 0
        width, height = 2 * np.sqrt(covariance)
    
    # Draw the Ellipse
    for nsig in range(1, 4):
        ax.add_patch(Ellipse(position, nsig * width, nsig * height, angle=angle, **kwargs))

def plot_discr(out, train_original, x_lim=[], y_lim=[], elipse=True):
    df_data = pd.DataFrame()
    df_generated = pd.DataFrame()
    
    df_data["x"] = out['Ez.y'][:,0]
    df_data["y"] = out['Ez.y'][:,1]
    
    zz = out['zz']
    
#     zz = famd.transform(pred_famd)
    
    df_generated["x"],  df_generated["y"] = zz[:,0], zz[:,1]
    
    df_generated["method"] = "synthetic (MIAMI)"
    
    df_data["method"] = train_original["label"]
    
    df_scatter = pd.concat([df_data, df_generated], axis=0, ignore_index=True)
    

    # Create a scatter plot
    fig, ax = plt.subplots(figsize=(8, 6))
    ax = sns.scatterplot(data=df_scatter, x="x", y="y", style="method")
    # Add a title and axis labels
    ax.set_title('Scatter plot grouped by type')
    ax.set_xlabel('PC1')
    ax.set_ylabel('PC2')

    # Create a legend
    legend = ax.legend(loc='center left', bbox_to_anchor=(1.05, 0.5), title="Data", borderaxespad=0.)
    if len(x_lim)>0:
        ax.set_xlim(x_lim[0], x_lim[1])
        ax.set_ylim(y_lim[0], y_lim[1])
        
    if elipse:
        weights_ = out['w_s']  
        means_ = out['mu'][0]
        covariances_ = out['sigma'][0]
        
        w_factor = 0.2 / weights_.max()
        for pos, covar, w in zip(means_, covariances_, weights_):
            draw_ellipse(pos, covar, alpha=w * w_factor)
    plt.show()

plot_discr(out, train_original)
# plot_discr(out, train_original, x_lim=x_lim, y_lim=y_lim)


In [ ]:
def check_constraints(y, var_distrib, le_dict, colnames, layer_height_cols, brace_cols, continuous_ranges=None):
    cont = y[var_distrib == 'continuous']
    
    
#     y_trans = le_dict[colname].inverse_transform(pred[colname].astype(int))
    
    if (cont<0).any():
        return False
    if continuous_ranges is not None:
        ranges = continuous_ranges[0]
        column_ranges = continuous_ranges[1]
        values = y[column_ranges]
        
        within_ranges = np.logical_and(values >= ranges[:, 0], values <= ranges[:, 1])

        if not within_ranges.all():
            return False
    if y[colnames == 'radius_bottom'] < y[colnames == 'radius_top']:
        # radius bottom bigger than radius top
        return False
    
    n_layers = int(y[colnames == 'n_layers'])
    n_layers_decode = int(le_dict['n_layers'].inverse_transform([n_layers])[0])
    n_layer_corrected = n_layers_decode - 2
    braces_corrected = n_layers_decode - 1
    relevant_brace = brace_cols[:braces_corrected]
    relevant_brace_cols = zip(y[relevant_brace], colnames[relevant_brace])
    transf_brace = [le_dict[brace_col_name].inverse_transform([int(brace_col)])[0] for brace_col, brace_col_name in relevant_brace_cols]
    if "nan" in transf_brace:
        return False
    
    if n_layer_corrected>0:
        layer_heights = y[layer_height_cols]
        relevant_layer_heights = layer_heights[:n_layer_corrected]
        
        if (relevant_layer_heights>=1).any():
            return False
        is_ascending = all(relevant_layer_heights[i] <= relevant_layer_heights[i + 1] for i in range(len(relevant_layer_heights) - 1))
        if not is_ascending:
            return False
    return True

In [ ]:
from src.oversample import draw_new_bin, draw_new_ord,\
                       draw_new_categ, draw_new_cont

def generate_pseudo(point, out, nj, num_points=10, r=0.5):  
    
    x_val = out['Ez.y'][point,0]
    y_val = out['Ez.y'][point,1]
    
    

    
    samples_total = 5*num_points # sample 5 times more than wanted because we loose some
    z = sample_points_in_circle(x_val, y_val, r, samples_total)
    
    lambda_bin = np.array(out['lambda_bin']) 
    lambda_ord = out['lambda_ord'] 
    lambda_categ = out['lambda_categ'] 
    lambda_cont = np.array(out['lambda_cont'])
    
    nj_bin = nj[pd.Series(var_distrib).isin(['bernoulli', 'binomial'])].astype(int)
    nj_ord = nj[var_distrib == 'ordinal'].astype(int)
    nj_categ = nj[var_distrib == 'categorical'].astype(int)

    y_std = out['y_std']
    
    y_bin_new = []
    y_categ_new = []
    y_ord_new = []
    y_cont_new = []


    y_bin_new.append(draw_new_bin(lambda_bin, z, nj_bin))
    y_categ_new.append(draw_new_categ(lambda_categ, z, nj_categ))
    y_ord_new.append(draw_new_ord(lambda_ord, z, nj_ord))
    y_cont_new.append(draw_new_cont(lambda_cont, z))
    # Stack the quantities
    y_bin_new = np.vstack(y_bin_new)
    y_categ_new = np.vstack(y_categ_new)
    y_ord_new = np.vstack(y_ord_new)
    y_cont_new = np.vstack(y_cont_new)

    # "Destandardize" the continous data
    y_cont_new = y_cont_new * y_std

    # Put them in the right order and append them to y
    type_counter = {'count': 0, 'ordinal': 0,\
                    'categorical': 0, 'continuous': 0} 

    y_new = np.full((samples_total, out['y_shape'][1]), np.nan)
    # Quite dirty:
    for j, var in enumerate(var_distrib):
        if (var == 'bernoulli') or (var == 'binomial'):
            y_new[:, j] = y_bin_new[:, type_counter['count']]
            type_counter['count'] =  type_counter['count'] + 1
        elif var == 'ordinal':
            y_new[:, j] = y_ord_new[:, type_counter[var]]
            type_counter[var] =  type_counter[var] + 1
        elif var == 'categorical':
            y_new[:, j] = y_categ_new[:, type_counter[var]]
            type_counter[var] =  type_counter[var] + 1
        elif var == 'continuous':
            y_new[:, j] = y_cont_new[:, type_counter[var]]
            type_counter[var] =  type_counter[var] + 1
        else:
            raise ValueError(var, 'Type not implemented')
    cols = out['cols']
    layer_cols = np.array([i for i, s in enumerate(cols) if s.startswith('layer_height')])
    brace_cols = np.array([i for i, s in enumerate(cols) if s.startswith('brace')])
    other_continuous_cols = list(set(np.where(var_distrib == 'continuous')[0]) - set(layer_cols))
    ranges_cont = [out['ranges'][other_continuous_cols], other_continuous_cols]
    
    mask = np.apply_along_axis(lambda x: check_constraints(x, var_distrib, le_dict, cols, layer_cols, brace_cols, continuous_ranges=ranges_cont), axis=1, arr=y_new)
    filtered_rows = y_new[mask]
    filtered_z = z[mask]
    filtered_rows = filtered_rows[:num_points,:]
    filtered_z = filtered_z[:num_points,:]
    return filtered_rows, filtered_z

In [ ]:
def sample_points_in_circle(center_x, center_y, radius, num_points):
    # Generate random polar coordinates
    theta = 2 * np.pi * np.random.rand(num_points)
    r = radius * np.sqrt(np.random.rand(num_points))
    
    # Convert polar coordinates to Cartesian coordinates
    x = center_x + r * np.cos(theta)
    y = center_y + r * np.sin(theta)
    
    return np.array(list(zip(x, y)))

def plot_near(z_points, item, x_lim=[], y_lim=[], elipse=True, save=False):
    df_data = pd.DataFrame()
    df_generated = pd.DataFrame()
    
    
    df_data["x"] = out['Ez.y'][:,0]
    df_data["y"] = out['Ez.y'][:,1]
    
    
#     zz = famd.transform(pred_famd)
    
    
    
    df_data["method"] = "real"
    df_data["size"] = 1
    
    df_scatter = pd.concat([df_data], axis=0, ignore_index=True)
    
    df_item = pd.concat([df_scatter.loc[item:item]], axis=0, ignore_index=True) 
    
    df_item["method"] = "selected"
    df_item["size"] = 2


    points = z_points
    df_points = pd.DataFrame(points, columns=["x", "y"])
    df_points["method"] = "sampled"
    df_points["size"] = 3
    
    # all_dfs = [df_scatter, df_item, df_points]
    all_dfs = [df_scatter, df_item, df_points]

    df_all = pd.concat(all_dfs, axis=0, ignore_index=True )

    # Create a scatter plot
    fig, ax = plt.subplots(figsize=(14, 8))
    # ax = sns.scatterplot(data=df_all, x="x", y="y", hue="method", style="method", size="method", sizes=(50,200))
    plt.scatter(df_scatter["x"], df_scatter["y"], marker="o", s=30, label="Real data", alpha=0.5)
    plt.scatter(df_item["x"], df_item["y"], marker="X", c="red", s=200, label="Selected instance", alpha=1)
    plt.scatter(df_points["x"], df_points["y"], marker="x", s=70, label="Sampled synthetic", alpha=1)
    # Add a title and axis labels
    # ax.set_title('Scatter plot grouped by type')
    ax.set_xlabel('Dim 0', fontsize=20)
    ax.set_ylabel('Dim 1', fontsize=20)

    methods = df_all["method"].unique()
    # Customize the legend labels
    # handles, labels = ax.get_legend_handles_labels()
    # indices = np.where(np.isin(labels, methods))[0].tolist()
    # handles = list(np.array(handles)[indices])
    # labels = list(np.array(labels)[indices])
    # handles = handles[1:4]
    # labels = labels[1:4]
    # plt.legend(loc='center left', bbox_to_anchor=(1.05, 0.5), title="Data", borderaxespad=0.)
    # plt.legend(fontsize=20, loc='center left', bbox_to_anchor=(1.05, 0.5))
    plt.legend(fontsize=20, loc='lower left')
    if len(x_lim)>0:
        ax.set_xlim(x_lim[0], x_lim[1])
        ax.set_ylim(y_lim[0], y_lim[1])
        
    if elipse:
        weights_ = out['w_s']  
        means_ = out['mu'][0]
        covariances_ = out['sigma'][0]
        
        w_factor = 0.2 / weights_.max()
        for pos, covar, w in zip(means_, covariances_, weights_):
            draw_ellipse(pos, covar, alpha=w * w_factor)

    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.title("Latent space", fontsize=20)
    if save:
        plt.savefig("sampling.png", dpi=300, bbox_inches="tight", pad_inches=0)
    plt.show()

    return points

num_points=20
POINT = 68
R = 0.6
# generated_close, z_points = generate_pseudo(POINT, out, nj, num_points=num_points, r=R)
z_points = plot_near(z_points, POINT, x_lim=[-2.5, 2.5], y_lim=[-2.5, 2.5], save=True)



In [ ]:
#================================================================
# Inverse transform the datasets
#================================================================
generated_close_df = pd.DataFrame(generated_close, columns = train.columns) 
generated_close_trans = generated_close_df.copy()

for j, colname in enumerate(train.columns):
    if colname in le_dict.keys():
        generated_close_trans[colname] = le_dict[colname].inverse_transform(generated_close_df[colname].astype(int))
    
generated_close_post = generated_close_trans.copy()    


layer_height_cols = generated_close_post.filter(like='layer_height').columns.tolist()

# Define a function to apply to each row
def replace_layer_height(row, brace_cols, layer_height_cols):
    n_layers = int(row['n_layers'])
    layers_set_one = layer_height_cols[n_layers-2:]
    braces_set_nan = brace_cols[n_layers-1:]
    # Set the layer_height columns to 1.0 for all rows after the nth row
    row[layers_set_one] = 1.0
    row[braces_set_nan] = "nan"
    
    # Return the modified row
    return row

# Apply the function to each row of the DataFrame
generated_close_post = generated_close_post.apply(lambda x: replace_layer_height(x, brace_cols, layer_height_cols), axis=1)


    
generated_close_post["label"] = "sampled"


points_post = pd.concat([generated_close_post, train_original[POINT:POINT+1]],ignore_index=True)
json_output = transform_df_to_json(points_post)
# with open(res_folder + "close_point_generated.json", "w") as json_file:
#     json.dump(json_output, json_file, indent=4)

In [ ]:
print(generated_close_post)

In [ ]:
train_original[POINT:POINT+1]

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
from src.MIAMI import get_samples

# Initialize lists to store results
target_values = range(100, 10001, 100)
num_iterations = 1  # Number of times to run the function for each target value
execution_times = []
lambda_bin, lambda_ord, lambda_categ, lambda_cont = out["lambda_bin"], out["lambda_ord"], out["lambda_categ"], out["lambda_cont"]
y_std = out["y_std"]
other_continuous_cols = list(set(np.where(var_distrib == 'continuous')[0]) - set(layer_cols))
ranges_cont = [out['ranges'][other_continuous_cols], other_continuous_cols]
w = out['best_w_s']
layer_cols__ = np.array([i for i, s in enumerate(cols) if s.startswith('layer_height')])
brace_cols__ = np.array([i for i, s in enumerate(cols) if s.startswith('brace')])
# for target_nb_pseudo_obs in target_values:
#     times_for_target = []
#     for iteration in range(num_iterations):
#         print(f"Processing target_nb_pseudo_obs = {target_nb_pseudo_obs}, Iteration {iteration+1}/{num_iterations}...")
#         start_time = time.time()
#         y_new_all = get_samples(train, target_nb_pseudo_obs, out, var_distrib, p, k, r, mu, sigma, w, lambda_bin, lambda_ord,\
#                 lambda_categ, lambda_cont, nj_bin, nj_ord, nj_categ, y_std, le_dict, cols, layer_cols__, brace_cols__, ranges_cont)
#         end_time = time.time()
#         execution_time = end_time - start_time
#         times_for_target.append(execution_time)

#     execution_times.append(times_for_target)

# # Calculate mean and standard deviation for each target value
# mean_execution_times = [np.mean(times) for times in execution_times]
# std_execution_times = [np.std(times) for times in execution_times]



In [ ]:
# import os
# def generate_seeds(target_nb_pseudo_obs, num_seeds=5):
#     for seed in range(num_seeds):
#         out_s = get_samples(train, target_nb_pseudo_obs, out, var_distrib, p, k, r, mu, sigma, w, lambda_bin, lambda_ord,\
#                 lambda_categ, lambda_cont, nj_bin, nj_ord, nj_categ, y_std, le_dict, cols, layer_cols__, brace_cols__, ranges_cont)        
#         pred = pd.DataFrame(out_s['y_all'], columns = train.columns) 
#         pred_post, pred_post_famd = post_process(pred, train.columns, le_dict, discrete_features, brace_cols, dtype, layer_cols_)
#         json_output = transform_df_to_json(pred_post)
#         outfolder = res_folder + "seeds/" + str(target_nb_pseudo_obs) +"/"
#         if not os.path.exists(outfolder):
#             os.makedirs(outfolder)
#         out_name = "dgmm("+"_".join(DATA_NAMES) + ")_" + str(target_nb_pseudo_obs) + "_seed_" + str(seed) + ".json"
#         output_filename = outfolder + out_name
#         with open(output_filename, "w") as json_file:
#             json.dump(json_output, json_file, indent=4)

# checkpoints = [100, 500, 1000, 1500, 2000]

# for checkpoint in checkpoints:
#     generate_seeds(checkpoint, 5)

In [ ]:
# # Save results to a file
# with open(res_folder+'execution_times.txt', 'w') as f:
#     for target_nb_pseudo_obs, mean_time, std_time in zip(target_values, mean_execution_times, std_execution_times):
#         f.write(f'Target obs: {target_nb_pseudo_obs}, Mean Time: {mean_time:.4f}, Std Dev: {std_time:.4f}\n')

In [ ]:
# # Plot the results with mean and std
# plt.plot(target_values, mean_execution_times, label='Mean Execution Time', color='b')
# plt.fill_between(target_values, 
#                  np.array(mean_execution_times) - np.array(std_execution_times), 
#                  np.array(mean_execution_times) + np.array(std_execution_times), 
#                  color='b', alpha=0.2, label='Standard Deviation')
# plt.xlabel('Target Pseudo Observations')
# plt.ylabel('Execution Time (seconds)')
# plt.title('Execution Time with Uncertainty vs. Target Pseudo Observations')
# plt.legend()
# plt.show()